# Convolutional Neural Network

### Importing the libraries

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [2]:
tf.__version__

'2.6.4'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [3]:
import os
os.makedirs('./test_data')

In [4]:
!pip install split_folders

In [5]:

import splitfolders # or import splitfolders
input_folder = "../input/car-vs-bike-classification-dataset/Car-Bike-Dataset"
output = "./test_data" #where you want the split datasets saved. one will be created if it does not exist or none is set

splitfolders.ratio(input_folder, output=output, seed=42, ratio=(.8, .1, .1)) # ratio of split are in order of train/val/test. You can change to whatever you want. For train/val sets only, you could do .75, .25 for example.

Copying files: 4000 files [00:16, 240.67 files/s]


In [6]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('../input/car-vs-bike-classification-dataset/Car-Bike-Dataset',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 4000 images belonging to 2 classes.


### Preprocessing the Test set

In [7]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('./test_data',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 4000 images belonging to 3 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [8]:
cnn = tf.keras.models.Sequential()

2022-11-07 10:48:37.757571: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-07 10:48:37.878539: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-07 10:48:37.879277: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-07 10:48:37.880558: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

### Step 1 - Convolution

In [9]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [10]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [11]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [12]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [13]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [14]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [15]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [16]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

2022-11-07 10:48:40.961594: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/25


2022-11-07 10:48:42.693383: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


 19/125 [===>..........................] - ETA: 11s - loss: 0.7171 - accuracy: 0.5428

/opt/conda/lib/python3.7/site-packages/PIL/Image.py:993: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


125/125 [==============================] - 35s 222ms/step - loss: 0.5051 - accuracy: 0.7340 - val_loss: 1.7808 - val_accuracy: 0.4200
Epoch 2/25
125/125 [==============================] - 28s 223ms/step - loss: 0.2807 - accuracy: 0.8827 - val_loss: 2.5965 - val_accuracy: 0.3750
Epoch 3/25
125/125 [==============================] - 28s 228ms/step - loss: 0.2490 - accuracy: 0.9022 - val_loss: 2.5537 - val_accuracy: 0.4155
Epoch 4/25
125/125 [==============================] - 28s 223ms/step - loss: 0.2393 - accuracy: 0.9045 - val_loss: 3.1289 - val_accuracy: 0.3745
Epoch 5/25
125/125 [==============================] - 28s 226ms/step - loss: 0.2085 - accuracy: 0.9162 - val_loss: 4.0818 - val_accuracy: 0.3658
Epoch 6/25
125/125 [==============================] - 29s 234ms/step - loss: 0.1974 - accuracy: 0.9175 - val_loss: 2.7282 - val_accuracy: 0.4420
Epoch 7/25
125/125 [==============================] - 30s 243ms/step - loss: 0.1918 - accuracy: 0.9185 - val_loss: 4.5538 - val_accuracy: 0.3

## Part 4 - Making a single prediction

In [31]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('../input/issacar/car.jfif', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'car'
else:
  prediction = 'bike'

In [32]:
print(prediction)

car
